# FAQ

This document will address frequently asked questions not addressed in other pages of the documentation.

## How do I install cobrapy?

Please see the [INSTALL.rst](https://github.com/opencobra/cobrapy/blob/stable/INSTALL.rst) file.

## How do I cite cobrapy?

Please cite the 2013 publication: [10.1186/1752-0509-7-74](http://dx.doi.org/doi:10.1186/1752-0509-7-74)

## How do I rename reactions or metabolites?

TL;DR Use `Model.repair` afterwards

When renaming metabolites or reactions, there are issues because cobra indexes based off of ID's, which can cause errors. For example:

In [1]:
import cobra.io
model = cobra.io.load_model("iYS1720")

for metabolite in model.metabolites:
    metabolite.id = f"test_{metabolite.id}"

try:
    model.metabolites.get_by_id(model.metabolites[0].id)
except KeyError as e:
    print(repr(e))

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


The Model.repair function will rebuild the necessary indexes

In [2]:
model.repair()
model.metabolites.get_by_id(model.metabolites[0].id)

Metabolite identifier,test_10fthf_c
Name,10-Formyltetrahydrofolate
Memory address,0x7f9c91ec0a90
Formula,C20H21N7O7
Compartment,c
In 9 reaction(s),"FTHFD, ULA4NFT, GARFT, AICART, TDPFRMT, MTHFC, BIOMASS_Ec_iAF1260_core_59p81M, BIOMASS_invivo, FMETTRS"


## How do I delete a gene?

That depends on what precisely you mean by delete a gene.

If you want to simulate the model with a gene knockout, use the `cobra.manipulation.knock_out_model_genes` function. The effects of this function are reversed when exiting a context.

In [3]:
model = cobra.io.load_model("iYS1720")
PGI = model.reactions.get_by_id("PGI")
print("bounds before knockout:", (PGI.lower_bound, PGI.upper_bound))
cobra.manipulation.knock_out_model_genes(model, ["STM4221"])
print("bounds after knockouts", (PGI.lower_bound, PGI.upper_bound))

bounds before knockout: (-1000.0, 1000.0)
bounds after knockouts (0.0, 0.0)


If you want to actually remove all traces of a gene from a model, this is more difficult because this will require changing all the `gene_reaction_rule` strings for reactions involving the gene.

## How do I change the reversibility of a Reaction?

`Reaction.reversibility` is a property in cobra which is computed when it is requested from the lower and upper bounds.

In [4]:
model = cobra.io.load_model("iYS1720")
model.reactions.get_by_id("PGI").reversibility

True

Trying to set it directly will result in an error or warning: 

In [5]:
try:
    model.reactions.get_by_id("PGI").reversibility = False
except Exception as e:
    print(repr(e))

/Users/uridavidakavia/PycharmProjects/cobrapy/src/cobra/core/reaction.py:810: UserWarning: Setting reaction reversibility is ignored
  warn("Setting reaction reversibility is ignored")


The way to change the reversibility is to change the bounds to make the reaction irreversible.

In [6]:
model.reactions.get_by_id("PGI").lower_bound = 10
model.reactions.get_by_id("PGI").reversibility

False

## How do I generate an LP file from a COBRA model?

### For optlang based solvers

With optlang solvers, the LP formulation of a model is obtained by its string representation. All solvers behave the same way.

In [7]:
with open('test.lp', 'w') as out:
    out.write(str(model.solver))

### How do I visualize my flux solutions?

Please browse the [visualization packages](https://opencobra.github.io/cobrapy/tags/visualization/) on our website for the most recent list of tools.